In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
import xgboost as xgb

pd.set_option('display.float_format',lambda x: '%.5f'%x)

print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path).fillna("")
dfPred= pd.read_csv(config.original_test_data_path).fillna("")
dfPred2= pd.read_csv(config.original_test_data2_path).fillna("")
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

dfTrain = dfTrain.rename(columns=col_name)
dfPred = dfPred.rename(columns=col_name)
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

dfAll = dfAll.replace({'':np.nan})
dfAll = dfAll.reset_index()

Load data...
Done.


In [8]:
dfAll = pd.read_csv('../../Data/All_non_missing.csv')
dfAll.shape

(721, 3231)

In [10]:
raw_predictors = []
for var in dfAll.columns:
    if not 'TOOL' in var and not 'ID' in var:
        raw_predictors.append(var)
raw_predictors

['210X1',
 '210X2',
 '210X3',
 '210X4',
 '210X5',
 '210X6',
 '210X7',
 '210X8',
 '210X9',
 '210X10',
 '210X11',
 '210X12',
 '210X13',
 '210X14',
 '210X15',
 '210X16',
 '210X18',
 '210X19',
 '210X20',
 '210X21',
 '210X23',
 '210X24',
 '210X25',
 '210X26',
 '210X27',
 '210X28',
 '210X29',
 '210X30',
 '210X32',
 '210X33',
 '210X34',
 '210X35',
 '210X36',
 '210X37',
 '210X39',
 '210X40',
 '210X41',
 '210X42',
 '210X43',
 '210X44',
 '210X45',
 '210X46',
 '210X47',
 '210X48',
 '210X49',
 '210X50',
 '210X51',
 '210X52',
 '210X53',
 '210X54',
 '210X55',
 '210X56',
 '210X57',
 '210X58',
 '210X59',
 '210X60',
 '210X61',
 '210X62',
 '210X63',
 '210X64',
 '210X65',
 '210X66',
 '210X67',
 '210X68',
 '210X69',
 '210X70',
 '210X71',
 '210X72',
 '210X73',
 '210X74',
 '210X75',
 '210X76',
 '210X77',
 '210X80',
 '210X81',
 '210X82',
 '210X83',
 '210X84',
 '210X85',
 '210X86',
 '210X87',
 '210X88',
 '210X89',
 '210X90',
 '210X91',
 '210X92',
 '210X93',
 '210X94',
 '210X95',
 '210X96',
 '210X97',
 '210X98

In [12]:
#Count 计算特定值在变量中重复出现的次数'''
dfAll = dfAll.replace({np.nan:-99999999})
'''added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    dfAll['cnt_'+var] = 0
    added.append('cnt_'+var)
    groups = dfAll.groupby([var])
    for name,group in groups:
        count = group[var].count()
        dfAll['cnt_'+var].ix[group.index] = count
dfAll[added].to_csv('../../Cache/Feat_cnt_col_non_missing.csv',index=False)'''

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close() 

added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]])
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['cnt_'+tool+'_'+var] = 0
        added.append('cnt_'+tool+'_'+var)
        for name,group in groups:
            grps = group.groupby([var])
            for name2,grp in grps:
                dfAll['cnt_'+tool+'_'+var].ix[grp.index] = float(len(grp))/float(len(group))
dfAll[added].to_csv('../../Cache/Feat_cnt_tool_non_missing.csv',index=False)

C:\Users\Leo Mao\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Leo Mao\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [13]:
#percentile 计算某一个值在全集中的位置（百分制）
dfAll = dfAll.replace({-99999999:np.nan})
added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    if 'TOOL' not in var:
        dfAll['pcent_'+var] = dfAll[var].rank(method='max')/float(len(dfAll))
        added.append('pcent_'+var)
dfAll[added].to_csv('../../Cache/Feat_pcent_col_non_missing.csv',index=False)
t = 0
added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]]) 
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['pcent_'+tool+'_'+var] = 0
        added.append('pcent_'+tool+'_'+var)
        for name,group in groups:
            dfAll['pcent_'+tool+'_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))
            
dfAll[added].to_csv('../../Cache/Feat_pcent_tool_non_missing.csv',index=False) 

C:\Users\Leo Mao\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Leo Mao\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
#do the variables minus when two variables' range overlap more than 50%
minus_var = ['ID']
for i in range(len(raw_predictors)-1):
    for j in range(i+1,len(raw_predictors)):
        if (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].max())*(dfAll[raw_predictors[i]].min()-dfAll[raw_predictors[j]].min())<0:
            if float(min(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min()))/max(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min())>=0.8:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)
        elif (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min())*(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())>0:
            if float(min(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())))/max(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min()))>=0.8:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)           

210X1-261X237
210X1-261X247
210X1-261X257
210X1-261X267
210X1-261X423
210X1-261X433
210X1-261X443
210X1-261X453
210X1-261X475
210X1-261X601
210X1-261X609
210X1-261X619
210X1-261X629
210X1-261X639
210X1-312X26
210X1-312X28
210X1-330X135
210X1-330X142
210X1-330X149
210X1-330X191
210X1-330X298
210X1-330X405
210X1-330X462
210X1-330X472
210X1-330X482
210X1-330X492
210X1-330X502
210X1-330X519
210X1-330X529
210X1-330X539
210X1-330X548
210X1-330X555
210X1-330X575
210X1-330X576
210X1-330X670
210X1-330X671
210X1-330X677
210X2-311X114
210X3-220X509
210X4-220X483
210X4-220X487
210X4-261X321
210X4-261X693
210X4-312X69
210X4-312X76
210X4-340X51
210X4-420X75
210X4-750X261
210X4-750X778
210X5-210X39
210X5-311X80
210X5-311X82
210X5-311X168
210X5-311X170
210X5-261X354
210X5-261X356
210X5-261X540
210X5-261X542
210X5-261X726
210X5-261X728
210X5-312X480
210X5-750X294
210X6-220X475
210X6-520X137
210X7-210X8
210X7-344X133
210X7-344X141
210X7-344X142
210X8-344X133
210X8-344X141
210X8-344X142
210X9-220X176
210

210X132-220X415
210X133-210X134
210X133-210X135
210X133-210X148
210X133-210X149
210X133-210X150
210X133-210X151
210X133-210X156
210X133-210X157
210X133-210X162
210X133-210X163
210X133-210X166
210X133-210X167
210X133-210X168
210X133-210X169
210X133-220X415
210X134-210X135
210X134-210X148
210X134-210X149
210X134-210X150
210X134-210X151
210X134-210X156
210X134-210X157
210X134-210X162
210X134-210X163
210X134-210X166
210X134-210X167
210X134-210X168
210X134-210X169
210X134-220X415
210X135-210X148
210X135-210X149
210X135-210X150
210X135-210X151
210X135-210X156
210X135-210X157
210X135-210X162
210X135-210X163
210X135-210X166
210X135-210X167
210X135-210X168
210X135-210X169
210X135-220X415
210X136-210X137
210X141-360X763
210X142-220X483
210X142-220X487
210X142-420X72
210X142-420X75
210X142-420X76
210X143-220X478
210X143-312X76
210X143-750X778
210X144-210X145
210X148-210X149
210X148-210X150
210X148-210X151
210X148-210X156
210X148-210X157
210X148-210X162
210X148-210X163
210X148-210X166
210X148-210X

220X163-310X152
220X164-220X168
220X164-220X177
220X164-310X148
220X164-310X150
220X164-310X152
220X165-220X166
220X166-310X22
220X167-344X198
220X167-344X199
220X168-220X177
220X168-310X148
220X168-310X150
220X168-310X152
220X169-310X153
220X170-310X159
220X171-220X176
220X171-310X158
220X171-520X323
220X171-520X324
220X171-520X325
220X171-520X326
220X171-520X327
220X172-220X179
220X172-220X217
220X172-310X154
220X172-310X155
220X172-420X114
220X172-420X116
220X172-420X118
220X172-520X305
220X172-520X306
220X172-520X307
220X172-520X308
220X172-520X309
220X172-520X329
220X172-520X330
220X172-520X331
220X172-520X333
220X175-220X176
220X175-310X154
220X175-310X155
220X175-310X158
220X176-310X154
220X176-310X155
220X176-310X158
220X176-520X332
220X177-310X148
220X177-310X150
220X177-310X152
220X178-220X183
220X178-310X167
220X178-360X57
220X179-310X154
220X179-310X158
220X179-310X163
220X179-520X332
220X180-310X164
220X180-311X115
220X181-220X500
220X181-310X161
220X181-310X162
220X181-31

220X462-312X487
220X462-400X228
220X463-220X465
220X463-220X540
220X463-360X1357
220X465-220X540
220X465-360X1357
220X470-311X69
220X470-311X157
220X470-261X343
220X470-261X529
220X470-261X715
220X470-312X315
220X471-220X476
220X471-220X480
220X471-311X4
220X471-312X97
220X475-520X137
220X476-220X480
220X476-311X4
220X476-520X432
220X477-312X70
220X477-312X71
220X477-312X77
220X477-312X81
220X477-312X82
220X477-312X83
220X477-312X331
220X477-312X332
220X477-312X334
220X477-312X335
220X477-340X99
220X477-360X712
220X480-311X4
220X480-520X432
220X481-220X486
220X481-220X488
220X481-220X489
220X481-311X43
220X481-330X906
220X481-330X1053
220X481-360X1002
220X481-400X56
220X481-750X1002
220X481-750X1164
220X482-330X527
220X482-330X537
220X482-330X546
220X482-420X80
220X483-220X487
220X483-420X72
220X483-420X75
220X483-420X76
220X484-312X253
220X484-312X350
220X484-312X351
220X484-312X352
220X484-312X377
220X484-340X85
220X486-220X488
220X486-220X489
220X486-311X43
220X486-330X906
220X486-3

311X59-311X60
311X59-311X61
311X59-311X62
311X59-311X63
311X59-311X64
311X59-261X331
311X59-261X332
311X59-261X333
311X59-261X334
311X59-261X335
311X59-261X336
311X59-261X337
311X59-261X338
311X59-261X517
311X59-261X518
311X59-261X519
311X59-261X520
311X59-261X521
311X59-261X522
311X59-261X523
311X59-261X524
311X59-261X703
311X59-261X704
311X59-261X705
311X59-261X706
311X59-261X707
311X59-261X708
311X59-261X709
311X59-261X710
311X60-311X61
311X60-311X62
311X60-311X63
311X60-311X64
311X60-261X331
311X60-261X332
311X60-261X333
311X60-261X334
311X60-261X335
311X60-261X336
311X60-261X337
311X60-261X338
311X60-261X517
311X60-261X518
311X60-261X519
311X60-261X520
311X60-261X521
311X60-261X522
311X60-261X523
311X60-261X524
311X60-261X703
311X60-261X704
311X60-261X705
311X60-261X706
311X60-261X707
311X60-261X708
311X60-261X709
311X60-261X710
311X61-311X62
311X61-311X63
311X61-311X64
311X61-261X331
311X61-261X332
311X61-261X333
311X61-261X334
311X61-261X335
311X61-261X336
311X61-261X337
311X61-

...

440AX11-440AX13
440AX11-440AX14
440AX11-440AX17
440AX11-440AX19
440AX11-440AX20
440AX11-440AX23
440AX11-440AX25
440AX11-440AX26
440AX11-440AX29
440AX11-440AX32
440AX11-440AX34
440AX11-440AX35
440AX11-440AX36
440AX11-440AX37
440AX11-440AX38
440AX11-440AX41
440AX11-440AX44
440AX11-440AX46
440AX11-440AX47
440AX11-440AX49
440AX11-440AX50
440AX12-440AX15
440AX12-440AX18
440AX12-440AX21
440AX12-440AX22
440AX12-440AX24
440AX12-440AX27
440AX12-440AX28
440AX12-440AX30
440AX12-440AX31
440AX12-440AX33
440AX12-440AX39
440AX12-440AX40
440AX12-440AX42
440AX12-440AX43
440AX12-440AX45
440AX12-440AX46
440AX12-440AX48
440AX12-440AX51
440AX13-440AX14
440AX13-440AX15
440AX13-440AX16
440AX13-440AX17
440AX13-440AX19
440AX13-440AX20
440AX13-440AX21
440AX13-440AX22
440AX13-440AX23
440AX13-440AX25
440AX13-440AX26
440AX13-440AX27
440AX13-440AX28
440AX13-440AX29
440AX13-440AX31
440AX13-440AX34
440AX13-440AX36
440AX13-440AX37
440AX13-440AX38
440AX13-440AX39
440AX13-440AX40
440AX13-440AX41
440AX13-440AX43
440AX13-

440AX58-440AX60
440AX58-440AX61
440AX58-440AX63
440AX58-440AX64
440AX58-440AX66
440AX58-440AX69
440AX58-440AX70
440AX58-440AX71
440AX58-440AX72
440AX58-440AX73
440AX58-440AX74
440AX58-440AX75
440AX58-440AX76
440AX58-440AX78
440AX58-440AX79
440AX58-440AX81
440AX58-440AX82
440AX58-440AX84
440AX58-440AX85
440AX58-440AX87
440AX58-440AX88
440AX58-440AX90
440AX58-440AX91
440AX58-440AX92
440AX58-440AX94
440AX58-440AX96
440AX58-440AX97
440AX58-440AX99
440AX58-440AX102
440AX59-440AX62
440AX59-440AX64
440AX59-440AX65
440AX59-440AX68
440AX59-440AX71
440AX59-440AX74
440AX59-440AX76
440AX59-440AX77
440AX59-440AX80
440AX59-440AX83
440AX59-440AX85
440AX59-440AX86
440AX59-440AX87
440AX59-440AX88
440AX59-440AX89
440AX59-440AX92
440AX59-440AX95
440AX59-440AX97
440AX59-440AX98
440AX59-440AX101
440AX60-440AX61
440AX60-440AX63
440AX60-440AX66
440AX60-440AX70
440AX60-440AX72
440AX60-440AX73
440AX60-440AX75
440AX60-440AX79
440AX60-440AX81
440AX60-440AX82
440AX60-440AX84
440AX60-440AX87
440AX60-440AX91
440AX6

440AX112-440AX115
440AX112-440AX122
440AX112-440AX125
440AX112-440AX134
440AX112-440AX139
440AX112-440AX145
440AX113-440AX123
440AX113-440AX126
440AX113-440AX132
440AX113-440AX137
440AX113-440AX143
440AX114-440AX135
440AX114-440AX138
440AX114-440AX144
440AX114-440AX147
440AX115-440AX122
440AX115-440AX124
440AX115-440AX125
440AX115-440AX134
440AX115-440AX145
440AX116-440AX121
440AX116-440AX128
440AX116-440AX139
440AX116-440AX149
440AX117-440AX119
440AX117-440AX128
440AX117-440AX129
440AX117-440AX137
440AX117-440AX140
440AX117-440AX143
440AX117-440AX150
440AX117-440AX152
440AX118-440AX130
440AX118-440AX151
440AX119-440AX129
440AX119-440AX137
440AX119-440AX140
440AX119-440AX150
440AX119-440AX152
440AX120-440AX138
440AX120-440AX141
440AX120-440AX153
440AX121-440AX149
440AX121-440AX154
440AX122-440AX125
440AX122-440AX134
440AX122-440AX145
440AX122-440AX146
440AX123-440AX126
440AX123-440AX129
440AX123-440AX132
440AX123-440AX137
440AX123-440AX143
440AX123-440AX150
440AX124-440AX125
440AX124-4

440AX192-440AX206
440AX193-440AX194
440AX193-440AX195
440AX193-440AX198
440AX193-440AX199
440AX193-440AX204
440AX193-440AX205
440AX193-440AX206
440AX194-440AX197
440AX194-440AX198
440AX194-440AX200
440AX194-440AX204
440AX194-440AX205
440AX194-440AX206
440AX195-440AX196
440AX195-440AX197
440AX195-440AX198
440AX195-440AX199
440AX195-440AX205
440AX196-440AX198
440AX196-440AX199
440AX197-440AX198
440AX197-440AX199
440AX197-440AX200
440AX197-440AX204
440AX197-440AX205
440AX197-440AX206
440AX198-440AX199
440AX198-440AX200
440AX198-440AX204
440AX198-440AX205
440AX200-440AX204
440AX200-440AX206
440AX201-440AX202
440AX201-440AX203
440AX202-440AX203
440AX204-440AX205
440AX204-440AX206
440AX205-440AX206
440AX207-440AX212
440AX211-440AX212
520X1-520X144
520X1-520X246
520X1-520X345
520X1-520X351
520X1-520X353
520X11-520X12
520X11-520X26
520X11-520X27
520X11-520X82
520X12-520X26
520X12-520X27
520X12-520X106
520X12-520X108
520X13-520X21
520X13-520X22
520X13-520X24
520X13-520X82
520X16-520X17
520X16-5

750X229-750X465
750X229-750X476
750X229-750X487
750X229-750X553
750X229-750X564
750X229-750X620
750X229-750X628
750X229-750X660
750X229-750X668
750X229-750X676
750X240-750X251
750X240-750X262
750X240-750X273
750X240-750X284
750X240-750X476
750X240-750X487
750X240-750X660
750X240-750X668
750X251-750X262
750X251-750X273
750X251-750X284
750X251-750X660
750X261-750X778
750X262-750X273
750X262-750X284
750X262-750X660
750X273-750X284
750X273-750X476
750X273-750X487
750X273-750X660
750X273-750X668
750X284-750X476
750X284-750X487
750X284-750X553
750X284-750X564
750X284-750X620
750X284-750X628
750X284-750X660
750X284-750X668
750X294-750X775
750X295-750X317
750X295-750X336
750X295-750X347
750X295-750X358
750X295-750X391
750X295-750X402
750X295-750X413
750X295-750X424
750X295-750X454
750X295-750X465
750X295-750X476
750X295-750X487
750X295-750X498
750X295-750X509
750X295-750X520
750X295-750X531
750X295-750X542
750X295-750X553
750X295-750X572
750X295-750X604
750X295-750X612
750X295-750X620
750X295-

In [16]:
dfAll[minus_var].to_csv('../../../Feat_minus_non_missing.csv')

In [10]:
dfAll = dfAll[minus_var+raw_predictors]
dfAll = dfAll.set_index(dfAll['ID'])
dfTrain_tmp = dfAll.loc[dfTrain['ID']]
dfTrain = pd.merge(dfTrain_tmp,dfTrain[['ID','Y']],'inner','ID')
dfPred = dfAll.loc[dfPred['ID']]

In [11]:
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')

In [12]:
f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

predictors=dfTrain.columns.tolist()[1:]
predictors.remove('Y')

predictors = predictors+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        try:
            predictors.remove(var)
        except:
            continue
    for var in var_change[key]['category']:
        try:
            predictors.remove(var)
        except:
            continue
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [17]:
from sklearn import metrics
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':3, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,
                         'colsample_bytree':0.6}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =100)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [24]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

[0]	train-rmse:2.33165	eval-rmse:2.33057
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:0.875801	eval-rmse:0.87615
[200]	train-rmse:0.349202	eval-rmse:0.363266
[300]	train-rmse:0.162023	eval-rmse:0.202634
[400]	train-rmse:0.098459	eval-rmse:0.165756
[500]	train-rmse:0.074039	eval-rmse:0.158725
[600]	train-rmse:0.059749	eval-rmse:0.157102
[700]	train-rmse:0.048807	eval-rmse:0.156246
[800]	train-rmse:0.040516	eval-rmse:0.155566
[900]	train-rmse:0.034075	eval-rmse:0.155343
Stopping. Best iteration:
[946]	train-rmse:0.031349	eval-rmse:0.155132

[0]	train-rmse:2.335	eval-rmse:2.31768
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:0.877115	eval-rmse:0.860831
[200]	train-rmse:0.350513	eval-rmse:0.345553
[300]	train-rmse:0.163366	eval-rmse:0.181203
[400]	train-rmse:0.0993

In [22]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)
imp.to_csv('../../Submission/imp_%s.csv'%today,header=False,index=False)
test_result.to_csv('../../Submission/test_%s.csv'%today,header=False,index=False)

In [ ]:
0.027994300372